## 感知器

In [1]:
# scikit-learn 单一 LTU 网络的 Perceptron

import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

iris = load_iris()
X = iris.data[:, (2, 3)] # petal length, petal width
y = (iris.target == 0).astype(np.int) # iris Setosa ?

per_clf = Perceptron(random_state=42)
per_clf.fit(X, y)

y_pred = per_clf.predict([[2, 0.5]])
print(y_pred)

[1]


**感知器和逻辑回归分类器相反，感知器不输出某个类的概率。它只能根据一个固定的阈值来做预测。因此更应该使用逻辑回归而非感知器。**

## 用 TensorFlow 的高级 API 来训练 MLP

In [2]:
import tensorflow as tf

feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(X)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300, 100], n_classes=10, feature_columns=feature_columns)
dnn_clf.fit(x=X, y=y, batch_size=50, steps=1000)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please specify feature columns explicitly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please access pandas data directly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please convert numpy dtypes explicitly.
Instructions for updating:
Please specify feature columns explicitly.
Instructions for updating:
Please switch to tf.contrib.estimator.*_head.
Instructions for updating:
Please replace uses of any Estimator from tf.contrib.learn with an Estimator from tf.estimator.*
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Using default config.
INFO:tensorf

INFO:tensorflow:loss = 0.0060908603, step = 201 (0.111 sec)
INFO:tensorflow:global_step/sec: 880.466
INFO:tensorflow:loss = 0.0028167455, step = 301 (0.114 sec)
INFO:tensorflow:global_step/sec: 874.606
INFO:tensorflow:loss = 0.0024324418, step = 401 (0.114 sec)
INFO:tensorflow:global_step/sec: 907.253
INFO:tensorflow:loss = 0.0027872128, step = 501 (0.110 sec)
INFO:tensorflow:global_step/sec: 841.263
INFO:tensorflow:loss = 0.001039065, step = 601 (0.119 sec)
INFO:tensorflow:global_step/sec: 795.096
INFO:tensorflow:loss = 0.000952394, step = 701 (0.126 sec)
INFO:tensorflow:global_step/sec: 701.183
INFO:tensorflow:loss = 0.000735834, step = 801 (0.143 sec)
INFO:tensorflow:global_step/sec: 814.928
INFO:tensorflow:loss = 0.0010076963, step = 901 (0.123 sec)
INFO:tensorflow:Saving checkpoints for 1000 into /var/folders/gt/gzycyzkn1tbg30xkr04j1g900000gn/T/tmp4p3tb8hn/model.ckpt.
INFO:tensorflow:Loss for final step: 0.0005541224.


DNNClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x10c1bdbe0>, 'hidden_units': [300, 100], 'feature_columns': (_RealValuedColumn(column_name='', dimension=2, default_value=None, dtype=tf.float64, normalizer=None),), 'optimizer': None, 'activation_fn': <function relu at 0x129a0e510>, 'dropout': None, 'gradient_clip_norm': None, 'embedding_lr_multipliers': None, 'input_layer_min_slice_size': None})

评估:

- 手动计算

```
from sklearn.metrics import accuracy_score

y_pred = list(dnn_clf.predict(X_test))
accuracy_score(y_test, y_pred)
```

- TF.Learn

```
dnn_clf.evaluate(X_test, y_test)
```

## 使用纯 TensorFlow 训练 DNN

In [3]:
import tensorflow as tf

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int64, shape=(None), name='y')

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name='weights')
        b = tf.Variable(tf.zeros([n_neurons]), name='biases')
        z = tf.matmul(X, W) + b
        if activation == 'relu':
            return tf.nn.relu(z)
        else:
            return z

In [4]:
with tf.name_scope('my_dnn'):
    hidden1 = neuron_layer(X, n_hidden1, 'hidden1', activation='relu')
    hidden2 = neuron_layer(hidden1, n_hidden2, 'hidden2', activation='relu')
    logits = neuron_layer(hidden2, n_outputs, 'outputs')

In [5]:
# 利用 tf 的 fully_connected() 来替换自己创建的 neuron_layer()
from tensorflow.contrib.layers import fully_connected

with tf.name_scope('dnn'):
    hidden1 = fully_connected(X, n_hidden1, scope='hidden1')
    hidden2 = fully_connected(hidden1, n_hidden2, scope='hidden2')
    logits = fully_connected(hidden2, n_outputs, scope='outputs', activation_fn=None)

In [6]:
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name='loss')

In [7]:
learning_rate = 0.01

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [8]:
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [9]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [10]:
# 执行阶段
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('data/')

n_epochs = 10
batch_size = 100

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        print(epoch, ' train_acc: ', acc_train, ' test_acc: ', acc_test)
    save_path = saver.save(sess, './model/mnist_model.ckpt')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
0  train_acc:  0.85  test_acc:  0.8786
1  train_acc:  0.92  test_acc:  0.9054
2  train_acc:  0.92  test_acc:  0.9129
3  train_acc:  0.92  test_acc:  0.9188
4  train_acc:  0.91  test_acc:  0.9254
5  train_acc:  0.91  test_acc:  0.9296
6  train_acc:  0.91  test_acc:  0.9332
7  train_acc:  0.93  test_acc:  0.937
8  train_acc:  0.93  test_acc:  0.9397
9  train_acc:  0.98  test_acc:  0.94

In [11]:
with tf.Session() as sess:
    saver.restore(sess, './model/mnist_model.ckpt')
    X_new_scaled = mnist.test.images[:20]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)
    print(y_pred)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./model/mnist_model.ckpt
[7 2 1 0 4 1 4 9 6 9 0 6 9 0 1 5 9 7 3 4]
